In [1]:
import numpy as np
import tensorflow as tf
import pickle
import json
from preprocess_v2 import *
import pandas as pd

In [2]:
def reverse_bias(content):
    """Convenience function with no generalization- just a hack to reverse the bias"""
    words = content.split()
    view = words[1]
    
    if view=='liberal':
        words[1] = 'conservative'
    else:
        words[1] = 'liberal'
    reverse_bias_content = ' '.join(words)
    return reverse_bias_content, view, words[1]



In [3]:
np.random.seed(2470)
train_content, train_title, test_content, test_title = train_test_split()

test_reverse_content = []
test_original_view = []
test_reverse_view = []

for content in test_content:
    reverse_bias_content, original_view, reverse_view = reverse_bias(content)
    
    test_reverse_content.append(reverse_bias_content)
    test_original_view.append(original_view)
    test_reverse_view.append(reverse_view)
    
(content_vocab, content_word_index, content_index_word, 
 title_vocab, title_word_index, title_index_word) = vectorize_data(train_content, train_title)

train_content_vec = CONTENT_VECTORIZER(train_content)
train_title_vec = TITLE_VECTORIZER(train_title)

test_content_vec = CONTENT_VECTORIZER(test_content)
test_reverse_content_vec = CONTENT_VECTORIZER(test_reverse_content)
test_title_vec = TITLE_VECTORIZER(test_title)


print(train_content_vec.shape, train_title_vec.shape, test_content_vec.shape, test_title_vec.shape,
      test_reverse_content_vec.shape)

glove_index = build_glove_embed_index()
title_embedding_init, title_vocab_size = build_embedding_init(title_word_index, glove_index)
content_embedding_init, content_vocab_size = build_embedding_init(content_word_index, glove_index)


2023-05-10 17:43:24.614557: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(25184, 256) (25184, 16) (1325, 256) (1325, 16) (1325, 256)
Unique words in glove: 400003
Hits: 14315; Misses: 685
Hits: 68712; Misses: 21651


In [4]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_size, window_size, initializer, trainable=False):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.window_size = window_size
        self.initializer = initializer
        self.trainable = trainable
        
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_size, mask_zero=True,
                                                   embeddings_initializer=self.initializer,
                                                   trainable=self.trainable)    
        self.positional_encoding = positional_encoding(window_size, embedding_size)
    
    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)
        
    def call(self, x, add_positional_embedding=True):
        length = tf.shape(x)[1]
        if add_positional_embedding:
            return self.embedding(x)+positional_encoding(length, self.embedding_size)
        else:
            return self.embedding(x)

class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()
        
class CrossAttention(BaseAttention):
    def call(self, x, context):
        attn_output = self.mha(query=x, key=context, value=context)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class GlobalSelfAttention(BaseAttention):
    def call(self, x, attention_mask=None):
        attn_output = self.mha(query=x, value=x, key=x, attention_mask=attention_mask)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class CausalSelfAttention(BaseAttention):
    def call(self, x, attention_mask=None):
        attn_output = self.mha(query=x, value=x, key=x, use_causal_mask=True, attention_mask=attention_mask)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x
    
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, embedding_size, ff_dim, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([tf.keras.layers.Dense(ff_dim, activation='relu'),
                                        tf.keras.layers.Dense(embedding_size),
                                        tf.keras.layers.Dropout(dropout_rate)])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, embedding_size, ff_dim, dropout_rate=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.embedding_size = embedding_size
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        
        self.self_attention = GlobalSelfAttention(num_heads=num_heads, key_dim=embedding_size,
                                                  dropout=dropout_rate)
        self.ffn = FeedForward(embedding_size, ff_dim)

    def call(self, x, attention_mask=None):
        x = self.self_attention(x, attention_mask=attention_mask)
        x = self.ffn(x)
        return x

class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, num_heads, ff_dim, vocab_size, embedding_size, 
                 window_size, embedding_initializer, embedding_trainability=False,  dropout_rate=0.1):
        super().__init__()

        self.num_layers = num_layers
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.window_size = window_size
        self.embedding_initializer = embedding_initializer
        self.embedding_trainability = embedding_trainability
        
        self.pos_embedding = PositionalEmbedding(self.vocab_size, self.embedding_size, self.window_size,
                                                 self.embedding_initializer, self.embedding_trainability)
        
        self.enc_layers = [EncoderLayer(num_heads, embedding_size, ff_dim, dropout_rate) for i in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x):
        # x is tokenized numerical values
        mask = self.pos_embedding.compute_mask(x)
        mask = mask[:,tf.newaxis,:]
        x = self.pos_embedding(x)
        
        # Add dropout.
        x = self.dropout(x)
        
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, attention_mask=mask)
        return x
    
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, embedding_size, ff_dim, dropout_rate=0.1):
        super(DecoderLayer, self).__init__()

        self.causal_self_attention = CausalSelfAttention(num_heads=num_heads, key_dim=embedding_size, 
                                                         dropout=dropout_rate)
        self.cross_attention = CrossAttention(num_heads=num_heads, key_dim=embedding_size, 
                                              dropout=dropout_rate)
        self.ffn = FeedForward(embedding_size, ff_dim)

    def call(self, x, context, attention_mask=None):
        x = self.causal_self_attention(x=x, attention_mask=attention_mask)
        x = self.cross_attention(x=x, context=context)
        x = self.ffn(x)
        return x
    
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, num_heads, ff_dim, vocab_size, embedding_size, window_size,
                 embedding_initializer, embedding_trainability=False, dropout_rate=0.1):
        
        super(Decoder, self).__init__()
        
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.window_size = window_size
        
        self.embedding_initializer = embedding_initializer
        self.embedding_trainability = embedding_trainability


        self.pos_embedding = PositionalEmbedding(self.vocab_size, self.embedding_size, self.window_size,
                                                 self.embedding_initializer, self.embedding_trainability)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        
        
        self.dec_layers = [DecoderLayer(num_heads, embedding_size, ff_dim,  dropout_rate=dropout_rate) 
                           for i in range(num_layers)]
        
    def call(self, x, context):
        # `x` is token-IDs shape (batch, target_seq_len)
        mask = self.pos_embedding.compute_mask(x)
        mask = mask[:,tf.newaxis,:]
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)
        x = self.dropout(x)

        for i in range(self.num_layers):
            x  = self.dec_layers[i](x, context, attention_mask=mask)

        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

class TransformerModel(tf.keras.Model):
    def __init__(self, num_layers, num_heads, ff_dim, embedding_size, 
                 content_vocab_size, title_vocab_size, content_window_size, title_window_size,
                 content_embedding_initializer, title_embedding_initializer,
                 content_embedding_trainability, title_embedding_trainability, 
                 dropout_rate=0.1):
        
        super().__init__()
        self.encoder = Encoder(num_layers, num_heads, ff_dim, content_vocab_size, embedding_size, 
                               content_window_size, content_embedding_initializer, content_embedding_trainability,
                               dropout_rate)
        self.decoder = Decoder(num_layers, num_heads, ff_dim, title_vocab_size, embedding_size,
                               title_window_size, title_embedding_initializer, title_embedding_trainability,
                               dropout_rate)
        
        self.dense_layer = tf.keras.layers.Dense(title_vocab_size)
    
    def call(self, inputs):
        content, title = inputs        
        context = self.encoder(content)
        output = self.decoder(title, context)
        logits = self.dense_layer(output)
        return logits

In [5]:
num_layers = 2
num_heads = 8
ff_dim = 256
embedding_size = GLOVE_EMBED_SZ
content_window_size = CONTENT_SEQ_LEN
title_window_size = TITLE_SEQ_LEN
content_embedding_initializer = tf.keras.initializers.Constant(content_embedding_init)
title_embedding_initializer = tf.keras.initializers.Constant(title_embedding_init)
content_embedding_trainability = True
title_embedding_trainability = True
dropout_rate = 0.1

train_title_labels = train_title_vec[:,:,tf.newaxis]
test_title_labels = test_title_vec[:,:,tf.newaxis]

model = TransformerModel(num_layers, num_heads, ff_dim, embedding_size, content_vocab_size, title_vocab_size,
                         content_window_size, title_window_size, content_embedding_initializer, title_embedding_initializer,
                         content_embedding_trainability, title_embedding_trainability, dropout_rate)

model.load_weights('../models/weights/modelv2-2blocks-8heads-256ffdim-trainableemb')


In [6]:
def sentence_from_ind(indexes, index_word_dict=title_index_word):
    sentence = ""
    
    for index in indexes:
   
        sentence += index_word_dict[index]
        sentence += " "
    
    return sentence



In [7]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu

def text_to_title(content, model=model, output_len=TITLE_SEQ_LEN):
    """Converts vectorized text to title
    Arguments:
        content - vectorized text"""
    
    start, end = tf.constant(title_word_index['<start>'], dtype=tf.int64), tf.constant(title_word_index['<end>'], dtype=tf.int64)
    start = start[tf.newaxis]
    end = end[tf.newaxis]
    
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(output_len):
        output = tf.transpose(output_array.stack())
        predictions = model([content[tf.newaxis], output], training=False)
        
        # Select the last token from the `seq_len` dimension.
        predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.
        predicted_id = tf.argmax(predictions, axis=2)

        # Concatenate the `predicted_id` to the output which is given to the
        # decoder as its input.
        output_array = output_array.write(i+1, predicted_id[0])

        if predicted_id == end:
            break
        
    output = output_array.stack().numpy().reshape(1,-1)
    predicted_title = sentence_from_ind(output[0].tolist())
    return predicted_title

true_titles = []
predicted_titles_original_bias = []
predicted_titles_reverse_bias = []
bleu_score_original_bias = []
bleu_score_reverse_bias = []

test_articles_len = len(test_content)

for index in range(test_articles_len):
    content_vec, reverse_content_vec, true_title = test_content_vec[index], test_reverse_content_vec[index], test_title[index]
    predicted_title_original_bias = text_to_title(content_vec)
    predicted_title_reverse_bias = text_to_title(reverse_content_vec)
    
    true_titles.append(true_title)
    predicted_titles_original_bias.append(predicted_title_original_bias)
    predicted_titles_reverse_bias.append(predicted_title_reverse_bias)
    
    bleu_score_original_bias.append(sentence_bleu([true_title.split()], predicted_title_original_bias.split(), 
                                    weights=(1,0,0,0)))
    bleu_score_reverse_bias.append(sentence_bleu([true_title.split()], predicted_title_reverse_bias.split(), 
                                    weights=(1,0,0,0)))
    
    
    if index%50==0:
        print(index)



/opt/miniconda3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/miniconda3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


0


/opt/miniconda3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300


In [8]:
df = pd.DataFrame(data=[true_titles, predicted_titles_original_bias, predicted_titles_reverse_bias,
                        bleu_score_original_bias, bleu_score_reverse_bias,
                        test_original_view[:test_articles_len], test_reverse_view[:test_articles_len]]).T
df.columns = ['true_title','predicted_title_original_bias', 'predicted_title_reverse_bias',
              'bleu_score_original_bias', 'bleu_score_reverse_bias',
              'original_view', 'reverse_view']
df['mean_bleu_score'] = (df['bleu_score_original_bias']+df['bleu_score_reverse_bias'])/2
df.sort_values(by=['mean_bleu_score'],ascending=[False],inplace=True)
df.to_csv('modelv2_results.csv', index=False)
